In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,43794
2,Huelva,Confirmados PDIA 14 días,1855
3,Huelva,Tasa PDIA 14 días,"361,4786522984586"
4,Huelva,Confirmados PDIA 7 días,828
5,Huelva,Tasa PDIA 7 dias,"161,3500399477756"
6,Huelva,Total Confirmados,43993
7,Huelva,Curados,38238
8,Huelva,Fallecidos,403


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  43794.0


/tmp/ipykernel_3300/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12191.0


/tmp/ipykernel_3300/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3300/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3300/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3300/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 730 personas en los últimos 7 días 

Un positivo PCR cada 300 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,43794.0,828.0,1855.0,513170.0,161.350040,361.478652,140.0
Huelva-Costa,25847.0,538.0,1171.0,289548.0,185.806844,404.423446,87.0
Condado-Campiña,13670.0,212.0,514.0,156231.0,135.696501,329.000006,42.0
Huelva (capital),12191.0,197.0,480.0,143837.0,136.960587,333.711076,33.0
Cartaya,1803.0,69.0,127.0,20083.0,343.574167,632.375641,15.0
Isla Cristina,2993.0,81.0,154.0,21393.0,378.628523,719.861637,14.0
Sierra de Huelva-Andévalo Central,3870.0,67.0,149.0,67391.0,99.419804,221.097773,11.0
Punta Umbría,1226.0,27.0,47.0,15355.0,175.838489,306.089222,8.0
Almonte,2210.0,40.0,69.0,24507.0,163.218672,281.552210,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,26.0,5.0,11.0,396.0,1262.626263,2777.777778,0.0
Linares de la Sierra,10.0,5.0,6.0,262.0,1908.396947,2290.076336,0.0
Alájar,34.0,4.0,14.0,759.0,527.009223,1844.532279,0.0
Almendro (El),49.0,6.0,10.0,840.0,714.285714,1190.476190,1.0
Rosal de la Frontera,61.0,11.0,15.0,1697.0,648.202711,883.912787,1.0
Escacena del Campo,223.0,15.0,18.0,2287.0,655.881067,787.057280,5.0
Isla Cristina,2993.0,81.0,154.0,21393.0,378.628523,719.861637,14.0
Ayamonte,1956.0,54.0,142.0,21104.0,255.875663,672.858226,7.0
Cartaya,1803.0,69.0,127.0,20083.0,343.574167,632.375641,15.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cortegana,284.0,1.0,11.0,4602.0,21.729683,239.026510,0.0,0.090909
Cumbres Mayores,129.0,1.0,9.0,1749.0,57.175529,514.579760,0.0,0.111111
Paterna del Campo,192.0,2.0,13.0,3457.0,57.853630,376.048597,1.0,0.153846
San Bartolomé de la Torre,355.0,3.0,14.0,3761.0,79.766020,372.241425,1.0,0.214286
Villablanca,266.0,3.0,13.0,2885.0,103.986135,450.606586,0.0,0.230769
Bonares,445.0,5.0,21.0,6060.0,82.508251,346.534653,1.0,0.238095
Higuera de la Sierra,57.0,1.0,4.0,1291.0,77.459334,309.837335,0.0,0.250000
Palos de la Frontera,984.0,7.0,28.0,11742.0,59.615057,238.460228,1.0,0.250000
Bollullos Par del Condado,1256.0,12.0,48.0,14387.0,83.408633,333.634531,3.0,0.250000
